<a href="https://colab.research.google.com/github/HeyJae-zero/Final-Team9/blob/main/%EC%8B%9C%EB%A6%AC%EC%A6%88%EB%AC%BC_%EB%BD%91%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN

# 1. 데이터 불러오기
# 파일 경로가 정확한지 다시 한번 확인해주세요.
try:
    df = pd.read_csv("/content/TMDB_processed_final.csv")
except FileNotFoundError:
    print("오류: 파일을 찾을 수 없습니다. 경로를 확인해주세요.")

# 'title' 컬럼에 비어있는 값(NaN)이 있을 경우를 대비해 빈 문자열로 채웁니다.
df['title'] = df['title'].fillna('')

print("데이터 불러오기 완료!")
print(f"전체 영화 개수: {len(df)}")


# 2. 텍스트 데이터 벡터화 (TF-IDF)
# 영화 제목(텍스트)을 컴퓨터가 이해할 수 있는 숫자 벡터로 변환합니다.
# ngram_range=(1, 3): 'Iron', 'Man', 'Iron Man' 처럼 단어 1개~3개 묶음을 모두 고려합니다.
# min_df=2: 최소 2개 이상의 문서(제목)에 나타나는 단어만 사용합니다. (너무 희귀한 단어 제외)
vectorizer = TfidfVectorizer(ngram_range=(1, 3), min_df=2, analyzer='word')
tfidf_matrix = vectorizer.fit_transform(df['title'])

print("TF-IDF 행렬 생성 완료!")
print(f"생성된 행렬의 크기: {tfidf_matrix.shape}") # (영화 개수, 생성된 단어/구문 개수)


# 3. 클러스터링 실행 (DBSCAN)
# DBSCAN은 스스로 군집의 개수를 찾아냅니다.
# eps: 클러스터로 묶을 최대 거리. 낮을수록 촘촘하게, 높을수록 널널하게 묶습니다. (튜닝 필요)
# min_samples: 하나의 클러스터를 구성하는 최소 영화(데이터) 개수.
clustering = DBSCAN(eps=0.5, min_samples=2, metric='cosine').fit(tfidf_matrix)

# 4. 결과 확인
# 생성된 클러스터 ID를 기존 데이터프레임에 새로운 컬럼으로 추가합니다.
df['cluster_id'] = clustering.labels_

print(f"\n총 {df['cluster_id'].max() + 1}개의 시리즈(클러스터)를 찾았습니다.")

# 클러스터별로 영화가 몇 개씩 포함되었는지 확인합니다.
# cluster_id가 -1인 경우는 어떤 클러스터에도 속하지 못한 '노이즈' 데이터입니다.
print("\n[클러스터별 영화 개수]")
print(df['cluster_id'].value_counts())

# -- 결과 상세히 살펴보기 --
# 노이즈(-1)를 제외하고, 각 클러스터에 어떤 영화들이 있는지 확인합니다.
print("\n[클러스터링 결과 상세]")
for cluster_num in sorted(df['cluster_id'].unique()):
    if cluster_num != -1: # 노이즈 데이터는 제외하고 출력
        print(f"\n===== 클러스터 #{cluster_num} =====")
        # 해당 클러스터에 속한 영화 제목들을 출력합니다.
        cluster_movies = df[df['cluster_id'] == cluster_num]['title']
        print(cluster_movies.to_string(index=False))

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
                                Muppets from Space
                            The Summit of the Gods
                      The House at the End of Time
                                      Dark Passage
                                             Ninja
                                         White God
                                     Double Dragon
                                     Certain Women
                                    A Mighty Heart
                                  Extreme Measures
                                   Chennai Express
                                         Meet Bill
                                            Barfi!
                                           The Ref
                                     Fritz the Cat
                                 Louder Than Bombs
                                   The Spacewalker
                                     Silver Streak
                                       Mary Re

In [ ]:
# ===== 2번 방법: 제목 접두어(prefix) 기반 시리즈 그룹핑 =====
# 전제: df 변수에 TMDB_processed_final.csv가 로드되어 있고, df['title'] 존재

import re
import pandas as pd

df = pd.read_csv('/content/TMDB_processed_final.csv')

# ---------------------------
# 설정
# ---------------------------
N_PREFIX = 2          # 접두어로 사용할 단어 수
MIN_GROUP_SIZE = 2    # 시리즈로 인정할 최소 편수
MIN_TITLE_LEN = 2     # 최소 단어 수
DROP_TAIL_PAREN = True # 괄호 부제 제거 여부

# ---------------------------
# 제목 정규화 함수
# ---------------------------
def normalize_title_for_prefix(title: str) -> str:
    if not isinstance(title, str):
        return ""
    t = title.strip()

    # 괄호로 끝나는 부제 제거
    if DROP_TAIL_PAREN:
        while True:
            t2 = re.sub(r"\s*\([^()]*\)\s*$", "", t).strip()
            if t2 == t:
                break
            t = t2

    # 특수문자 제거
    t = re.sub(r"[^\w\s]", " ", t)
    # 다중 공백 축소
    t = re.sub(r"\s{2,}", " ", t)
    return t.strip()

def get_prefix_tokens(title: str, n_prefix: int = 2, min_title_len: int = 2) -> str:
    t = normalize_title_for_prefix(title)
    if not t:
        return ""
    tokens = t.split()
    if len(tokens) < min_title_len:
        return ""
    prefix = " ".join(tokens[:n_prefix]).lower()
    return prefix

# ---------------------------
# 접두어 생성
# ---------------------------
df = df.copy()
df['series_prefix'] = df['title'].fillna("").apply(lambda x: get_prefix_tokens(x, N_PREFIX, MIN_TITLE_LEN))
df['series_prefix'] = df['series_prefix'].replace("", pd.NA)

# ---------------------------
# 그룹 크기 계산 및 시리즈 판정
# ---------------------------
prefix_counts = df['series_prefix'].value_counts(dropna=True)
valid_prefixes = set(prefix_counts[prefix_counts >= MIN_GROUP_SIZE].index)

df['is_series_by_prefix'] = df['series_prefix'].isin(valid_prefixes)

# ---------------------------
# 시리즈 그룹 요약 테이블
# ---------------------------
series_groups_df = (
    df[df['is_series_by_prefix']]
    .groupby('series_prefix', as_index=False)
    .agg(
        n_titles=('title', 'count'),
        titles=('title', lambda x: x.dropna().tolist())
    )
    .sort_values(['n_titles', 'series_prefix'], ascending=[False, True])
    .reset_index(drop=True)
)

series_groups_df['series_name'] = series_groups_df['series_prefix']

# ---------------------------
# 원본 df에 시리즈명 매핑
# ---------------------------
prefix_to_name = dict(zip(series_groups_df['series_prefix'], series_groups_df['series_name']))
df['series_name'] = df['series_prefix'].map(prefix_to_name)

# ---------------------------
# 결과 확인
# ---------------------------
print("시리즈로 판정된 영화 수:", int(df['is_series_by_prefix'].sum()))
print("시리즈 그룹 수:", len(series_groups_df))
print(series_groups_df.head(10))

시리즈로 판정된 영화 수: 2034
시리즈 그룹 수: 678
    series_prefix  n_titles  \
0        the last        39   
1         the man        20   
2         the big        17   
3          in the        15   
4       the great        15   
5  the adventures        14   
6       star trek        13   
7      the secret        13   
8        the good        11   
9       the house        11   

                                              titles     series_name  
0  [The Last Samurai, The Last Airbender, The Las...        the last  
1  [The Man from U.N.C.L.E., The Man in the Iron ...         the man  
2  [The Big Lebowski, The Big Short, The Big Sick...         the big  
3  [In the Heart of the Sea, In the Name of the F...          in the  
4  [The Great Gatsby, The Great Wall, The Great D...       the great  
5  [The Adventures of Tintin, The Adventures of S...  the adventures  
6  [Star Trek, Star Trek Into Darkness, Star Trek...       star trek  
7  [The Secret Life of Pets, The Secret Life of W...    

In [ ]:
# ===== 시리즈물 정밀 추출 & 그룹핑 (hybrid rule-based) =====
# 전제: df 는 TMDB_processed_final.csv 가 로드된 DataFrame이며, 최소 ['title','release_date','director','cast','keywords'] 보유

import pandas as pd
import numpy as np
import re
from collections import defaultdict
from itertools import combinations

df = pd.read_csv('/content/TMDB_processed_final.csv')

# -----------------------------
# 1) 유틸 함수
# -----------------------------
STOPWORDS = set("""a an and the of to in on at for from with by or as into about over under after before
part episode chapter pt vol volume saga story legend rise return revenge war age dawn day night vs versus
don t s a an the of and""".split())
ROMAN = {"i","ii","iii","iv","v","vi","vii","viii","ix","x"}

def normalize_title(t: str) -> str:
    if not isinstance(t, str):
        return ""
    t = t.lower().strip()
    t = re.sub(r"\s*\([^()]*\)\s*$", "", t)           # 끝 괄호 부제 제거
    t = re.sub(r"[^\w\s:\-]", " ", t)                 # 문장부호 정리(콜론/대시는 유지)
    t = re.sub(r"\s{2,}", " ", t).strip()
    return t

def tokenize(t: str):
    return [w for w in re.split(r"[\s:\-]+", t) if w]

def remove_leading_numbers(tokens):
    i = 0
    while i < len(tokens) and (tokens[i].isdigit() or tokens[i] in ROMAN):
        i += 1
    return tokens[i:]

def sig_tokens(tokens, max_k=None):
    sig = [w for w in tokens if (w not in STOPWORDS and not w.isdigit() and w not in ROMAN)]
    return sig[:max_k] if max_k else sig

def anchor2(title):
    """제목 전체에서 유의미한 앞 2토큰(예: 'harry potter', 'fast furious')"""
    t = normalize_title(title)
    toks = remove_leading_numbers(tokenize(t))
    sig = sig_tokens(toks)
    return " ".join(sig[:2]) if len(sig) >= 2 else ""

def base3(title):
    """콜론/대시 앞의 첫 세그먼트에서 유의미한 앞 3토큰(예: 'kung fu panda', 'captain america')"""
    t = normalize_title(title)
    first_seg = re.split(r"[:\-]", t, maxsplit=1)[0].strip()
    toks = remove_leading_numbers(tokenize(first_seg))
    sig = sig_tokens(toks)
    return " ".join(sig[:3]) if sig else ""

def has_seq_suffix(title):
    """제목 끝의 숫자/로마숫자/Part/Chapter/Episode/Vol 패턴"""
    t = normalize_title(title)
    if re.search(r"(part|chapter|episode|pt|vol|volume)\s*(\d+|i|ii|iii|iv|v|vi|vii|viii|ix|x)\s*$", t):
        return True
    if re.search(r"\b(\d+|i|ii|iii|iv|v|vi|vii|viii|ix|x)\s*$", t):
        return True
    return False

def year_from_date(s):
    try:
        return int(str(s)[:4])
    except:
        return np.nan

def parse_list(s, max_n=None):
    if not isinstance(s, str):
        return []
    items = [x.strip() for x in s.split(",") if x.strip()]
    return items[:max_n] if max_n else items

def jacc(a, b):
    sa, sb = set(a), set(b)
    if not sa or not sb:
        return 0.0
    return len(sa & sb) / len(sa | sb)

# -----------------------------
# 2) 특징 만들기
# -----------------------------
df = df.copy()
df['year'] = df['release_date'].apply(year_from_date)
df['anchor2'] = df['title'].fillna("").apply(anchor2)
df['base3']   = df['title'].fillna("").apply(base3)
df['seq_suffix'] = df['title'].fillna("").apply(has_seq_suffix)
df['cast_list']  = df['cast'].apply(lambda s: parse_list(s, max_n=6))
df['lead3']      = df['cast'].apply(lambda s: parse_list(s, max_n=3))
df['keywords_list'] = df['keywords'].apply(lambda s: [w.strip().lower() for w in s.split(",")] if isinstance(s,str) else [])

df.replace({"anchor2": {"": np.nan}, "base3": {"": np.nan}}, inplace=True)

# -----------------------------
# 3) 같은 anchor2 내에서 “증거 점수”로 엣지 생성
# -----------------------------
groups = defaultdict(list)
for idx, row in df.dropna(subset=['anchor2']).iterrows():
    groups[row['anchor2']].append(idx)

def qualify_pair(i, j):
    a = df.loc[i]; b = df.loc[j]
    if a['anchor2'] != b['anchor2']:
        return False, 0.0

    base_match   = (a['base3'] == b['base3'])         # 같은 첫 세그먼트(예: 'kung fu panda')
    same_director = isinstance(a['director'], str) and a['director'] != "" and a['director'] == b['director']
    cast_sim     = jacc(a['cast_list'], b['cast_list'])
    lead_overlap = len(set(a['lead3']) & set(b['lead3'])) >= 1    # 톱3 배우 중 1명이라도 겹치면 강한 신호
    keyw_sim     = jacc(a['keywords_list'], b['keywords_list'])
    ygap         = abs((a['year'] if pd.notna(a['year']) else 0) - (b['year'] if pd.notna(b['year']) else 0))
    has_sequel   = a['seq_suffix'] or b['seq_suffix']

    # 점수화(가중치는 정밀도 중심)
    score = 0.0
    if has_sequel:      score += 0.50
    if same_director:   score += 0.30
    if cast_sim >= 0.25: score += 0.25
    if lead_overlap:    score += 0.25
    if keyw_sim >= 0.50: score += 0.15
    if pd.notna(a['year']) and pd.notna(b['year']) and ygap <= 12: score += 0.10

    # base3가 다르면 임계치를 더 엄격히(다른 프랜차이즈 혼선 방지: 예 'kung fu panda' vs 'kung fu hustle')
    threshold = 0.80 if base_match else 0.95
    return score >= threshold, score

edges = []
for key, idxs in groups.items():
    if len(idxs) < 2:
        continue
    for i, j in combinations(idxs, 2):
        ok, _ = qualify_pair(i, j)
        if ok:
            edges.append((i, j))

# -----------------------------
# 4) Union-Find로 연결 성분 = 시리즈 그룹
# -----------------------------
parent = {}
def find(x):
    parent.setdefault(x, x)
    if parent[x] != x:
        parent[x] = find(parent[x])
    return parent[x]
def union(a, b):
    ra, rb = find(a), find(b)
    if ra != rb:
        parent[rb] = ra

for i, j in edges:
    union(i, j)

from collections import defaultdict
comp = defaultdict(list)
for idx in set([k for e in edges for k in e]):
    comp[find(idx)].append(idx)

# -----------------------------
# 5) 결과 테이블(시리즈 요약) + 원본 라벨 부여
# -----------------------------
rows = []
for r, idxs in comp.items():
    titles = df.loc[idxs, 'title'].tolist()
    years  = df.loc[idxs, 'year'].dropna().astype(int).tolist()
    name   = df.loc[idxs, 'anchor2'].value_counts().idxmax()
    rows.append({
        "series_key": r,
        "series_name": name,
        "n_titles": len(idxs),
        "years_min": min(years) if years else np.nan,
        "years_max": max(years) if years else np.nan,
        "titles": sorted(titles)
    })

series_df = pd.DataFrame(rows).sort_values(["n_titles","series_name"], ascending=[False, True]).reset_index(drop=True)

# 원본 df에 라벨 부여
in_series_titles = set(t for r in rows for t in r["titles"])
df['is_series']   = df['title'].isin(in_series_titles)
name_map = {}
for r in rows:
    for t in r["titles"]:
        name_map[t] = r["series_name"]
df['series_group'] = df['title'].map(name_map)

# -----------------------------
# 6) 사용 예
# -----------------------------
print("시리즈 후보 엣지 수:", len(edges))
print("감지된 시리즈 그룹 수:", len(series_df))
print(series_df[["series_name","n_titles","years_min","years_max"]])

# 저장 옵션
# series_df.to_csv("series_groups_hybrid.csv", index=False)
# df.to_csv("TMDB_with_series_labels.csv", index=False)

시리즈 후보 엣지 수: 249
감지된 시리즈 그룹 수: 133
            series_name  n_titles  years_min  years_max
0          harry potter         8       2001       2011
1    mission impossible         7       1996       2023
2            death wish         5       1974       1994
3          fast furious         5       2001       2013
4           scary movie         5       2000       2013
..                  ...       ...        ...        ...
128           winx club         2       2010       2014
129          wolf creek         2       2005       2013
130        wonder woman         2       2017       2020
131          work glory         2       2005       2006
132          young guns         2       1988       1990

[133 rows x 4 columns]


In [ ]:
# ===== 시리즈물 추출 & 그룹핑 (하이브리드 규칙 + 쉼표 split 적용) =====
# 전제: df = pd.read_csv("TMDB_processed_final.csv") 완료됨
# 필요 컬럼: ['title','release_date','director','cast','keywords']

import pandas as pd
import numpy as np
import re
from collections import defaultdict
from itertools import combinations

# -----------------------------
# 1) 유틸 함수
# -----------------------------
STOPWORDS = set("""a an and the of to in on at for from with by or as into about over under after before
part episode chapter pt vol volume saga story legend rise return revenge war age dawn day night vs versus
don t s a an the of and""".split())
ROMAN = {"i","ii","iii","iv","v","vi","vii","viii","ix","x"}

def normalize_title(t: str) -> str:
    if not isinstance(t, str):
        return ""
    t = t.lower().strip()
    t = re.sub(r"\s*\([^()]*\)\s*$", "", t)           # 끝 괄호 부제 제거
    t = re.sub(r"[^\w\s:\-]", " ", t)                 # 문장부호 정리(콜론/대시는 유지)
    t = re.sub(r"\s{2,}", " ", t).strip()
    return t

def tokenize(t: str):
    return [w for w in re.split(r"[\s:\-]+", t) if w]

def remove_leading_numbers(tokens):
    i = 0
    while i < len(tokens) and (tokens[i].isdigit() or tokens[i] in ROMAN):
        i += 1
    return tokens[i:]

def sig_tokens(tokens, max_k=None):
    sig = [w for w in tokens if (w not in STOPWORDS and not w.isdigit() and w not in ROMAN)]
    return sig[:max_k] if max_k else sig

def anchor2(title):
    """제목 전체에서 유의미한 앞 2토큰"""
    t = normalize_title(title)
    toks = remove_leading_numbers(tokenize(t))
    sig = sig_tokens(toks)
    return " ".join(sig[:2]) if len(sig) >= 2 else ""

def base3(title):
    """콜론/대시 앞의 첫 세그먼트에서 유의미한 앞 3토큰"""
    t = normalize_title(title)
    first_seg = re.split(r"[:\-]", t, maxsplit=1)[0].strip()
    toks = remove_leading_numbers(tokenize(first_seg))
    sig = sig_tokens(toks)
    return " ".join(sig[:3]) if sig else ""

def has_seq_suffix(title):
    """제목 끝의 숫자/로마숫자/Part/Chapter/Episode/Vol 패턴"""
    t = normalize_title(title)
    if re.search(r"(part|chapter|episode|pt|vol|volume)\s*(\d+|i|ii|iii|iv|v|vi|vii|viii|ix|x)\s*$", t):
        return True
    if re.search(r"\b(\d+|i|ii|iii|iv|v|vi|vii|viii|ix|x)\s*$", t):
        return True
    return False

def year_from_date(s):
    try:
        return int(str(s)[:4])
    except:
        return np.nan

# === 여기서 배우/키워드 split ===
def parse_csv_list(s, max_n=None, lower=True):
    """쉼표 기준 split → strip → lower → 리스트"""
    if not isinstance(s, str):
        return []
    items = [x.strip() for x in s.split(",") if x.strip()]
    if lower:
        items = [x.lower() for x in items]
    return items[:max_n] if max_n else items

def jacc(a, b):
    A, B = set(a), set(b)
    if not A or not B:
        return 0.0
    return len(A & B) / len(A | B)

# -----------------------------
# 2) 특징 생성
# -----------------------------
df = df.copy()
df['year'] = df['release_date'].apply(year_from_date)
df['anchor2'] = df['title'].fillna("").apply(anchor2)
df['base3']   = df['title'].fillna("").apply(base3)
df['seq_suffix'] = df['title'].fillna("").apply(has_seq_suffix)

# 배우: 상위 5명, lead3: 상위 3명
df['cast_list'] = df['cast'].apply(lambda s: parse_csv_list(s, max_n=5))
df['lead3']     = df['cast'].apply(lambda s: parse_csv_list(s, max_n=3))
# 키워드: 전체 사용
df['keywords_list'] = df['keywords'].apply(lambda s: parse_csv_list(s))

df.replace({"anchor2": {"": np.nan}, "base3": {"": np.nan}}, inplace=True)

# -----------------------------
# 3) 같은 anchor2 내에서 점수 계산
# -----------------------------
groups = defaultdict(list)
for idx, row in df.dropna(subset=['anchor2']).iterrows():
    groups[row['anchor2']].append(idx)

def qualify_pair(i, j):
    a = df.loc[i]; b = df.loc[j]
    if a['anchor2'] != b['anchor2']:
        return False, 0.0

    base_match   = (a['base3'] == b['base3'])
    same_director = isinstance(a['director'], str) and a['director'] != "" and a['director'] == b['director']
    cast_sim     = jacc(a['cast_list'], b['cast_list'])
    lead_overlap = len(set(a['lead3']) & set(b['lead3'])) >= 1
    keyw_sim     = jacc(a['keywords_list'], b['keywords_list'])
    ygap         = abs((a['year'] if pd.notna(a['year']) else 0) - (b['year'] if pd.notna(b['year']) else 0))
    has_sequel   = a['seq_suffix'] or b['seq_suffix']

    # 점수화
    score = 0.0
    if has_sequel:       score += 0.50
    if same_director:    score += 0.30
    if cast_sim >= 0.25: score += 0.25
    if lead_overlap:     score += 0.25
    if keyw_sim >= 0.50: score += 0.15
    if pd.notna(a['year']) and pd.notna(b['year']) and ygap <= 12: score += 0.10

    threshold = 0.80 if base_match else 0.95
    return score >= threshold, score

edges = []
for key, idxs in groups.items():
    if len(idxs) < 2:
        continue
    for i, j in combinations(idxs, 2):
        ok, _ = qualify_pair(i, j)
        if ok:
            edges.append((i, j))

# -----------------------------
# 4) Union-Find로 그룹핑
# -----------------------------
parent = {}
def find(x):
    parent.setdefault(x, x)
    if parent[x] != x:
        parent[x] = find(parent[x])
    return parent[x]
def union(a, b):
    ra, rb = find(a), find(b)
    if ra != rb:
        parent[rb] = ra

for i, j in edges:
    union(i, j)

comp = defaultdict(list)
for idx in set([k for e in edges for k in e]):
    comp[find(idx)].append(idx)

# -----------------------------
# 5) 결과 DataFrame
# -----------------------------
rows = []
for r, idxs in comp.items():
    titles = df.loc[idxs, 'title'].tolist()
    years  = df.loc[idxs, 'year'].dropna().astype(int).tolist()
    name   = df.loc[idxs, 'anchor2'].value_counts().idxmax()
    rows.append({
        "series_key": r,
        "series_name": name,
        "n_titles": len(idxs),
        "years_min": min(years) if years else np.nan,
        "years_max": max(years) if years else np.nan,
        "titles": sorted(titles)
    })

series_df = pd.DataFrame(rows).sort_values(["n_titles","series_name"], ascending=[False, True]).reset_index(drop=True)

# 원본 df에 라벨 부여
in_series_titles = set(t for r in rows for t in r["titles"])
df['is_series']   = df['title'].isin(in_series_titles)
name_map = {}
for r in rows:
    for t in r["titles"]:
        name_map[t] = r["series_name"]
df['series_group'] = df['title'].map(name_map)

# -----------------------------
# 6) 사용 예
# -----------------------------
print("감지된 시리즈 그룹 수:", len(series_df))
print(series_df.head(15)[["series_name","n_titles","years_min","years_max"]])

# series_df.to_csv("series_groups_final.csv", index=False)
# df.to_csv("TMDB_with_series_labels.csv", index=False)

감지된 시리즈 그룹 수: 133
           series_name  n_titles  years_min  years_max
0         harry potter         8       2001       2011
1   mission impossible         7       1996       2023
2           death wish         5       1974       1994
3         fast furious         5       2001       2013
4          scary movie         5       2000       2013
5    final destination         4       2000       2009
6         hunger games         4       2012       2015
7               ip man         4       2008       2019
8            john wick         4       2014       2023
9        lethal weapon         4       1987       1998
10             mad max         4       1979       2015
11             ah boys         3       2012       2015
12             ant man         3       2015       2023
13         back future         3       1985       1990
14            bad boys         3       1995       2020
